In [13]:
def handle_pulse(module, pulse, source):
    res = []
    match module[0]:
        case 'b':
            for dest in module[2]:
                res.append((dest,pulse))
        case '%':
            if not pulse:
                if module[2]:
                    module[2] = 0
                    for dest in module[1]:
                        res.append((dest,0))
                else:
                    module[2] = 1
                    for dest in module[1]:
                        res.append((dest,1))
        case '&':
            module[2][source] = pulse
            if all(module[2].values()):
                for dest in module[1]:
                    res.append((dest,0))
            else:
                for dest in module[1]:
                    res.append((dest,1))
    return res

In [14]:
modules = []

with open('input.txt') as f:
    for line in f:
        name, outputs = line[:-1].split(' -> ')
        m_ytpe = name[0]
        m_name = name[1:]
        m_outputs = outputs.split(', ')

        modules.append([m_ytpe, m_name, m_outputs])

conj = {}
flipflops = {}
for i,m in enumerate(modules):
    if m[0] == '&':
        conj[m[1]] = [m[0], m[2], {}]
    elif m[0] == '%':
        flipflops[m[1]] = [m[0], m[2], 0]

for n,c in conj.items():
    for m in modules:
        if n in m[2]:
            conj[n][2][m[1]] = 0

In [15]:
import math

low = 0
high = 0
button_presses = 0
lcms = {i:0 for i in conj['nc'][2]}

while True:
    if button_presses == 1000:
        print('Part 1:', low*high)
    button_presses += 1
    
    if all(lcms.values()):
        print('Part 2:', math.lcm(*lcms.values()))
        break

    cur_module = modules[0]
    low += 1
    pulse_stack = list(map(lambda x: (cur_module[1],*x),handle_pulse(cur_module,0,'button')))

    while pulse_stack:
        p = pulse_stack.pop(0)

        if p[0] in lcms and p[2]:
            if not lcms[p[0]]:
                lcms[p[0]] = button_presses

        if p[2]:
            high += 1
        else:
            low += 1

        if p[1] in flipflops:
            pulse_stack += list(map(lambda x: (p[1],*x),handle_pulse(flipflops[p[1]],p[2],p[0])))
        elif p[1] in conj:
            pulse_stack += list(map(lambda x: (p[1],*x),handle_pulse(conj[p[1]],p[2],p[0])))

Part 1: 1020211150
Part 2: 238815727638557
